In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf

# Model 2 - Mixed Effects Modelling
Main Idea: For model 1, we took every observation as i.i.d. However, there may be other effects that we haven't measured in our model such as team effects or even player effects. Our data consists of Squad Size and Average Market Value of each team both of which are Normalized by Average. There is 1 data point for each team for each year. This means that there are team effects and possibly year effects that we can encounter. The team effect is the first and main focus since some teams may have special patterns and trends. For example, if a team finishes 18, 19, or 20th in the league they are relegated to the EFL Champsionship (the 2nd division of English Football). This means that there are only a handful of teams that have remained in the Premier League since its inception (1992-93). This team effect may be crucial!

Resources/Links for Inspiration and Learning
- https://meghan.rbind.io/blog/2022-06-28-a-beginners-guide-to-mixed-effects-models/
- https://mfviz.com/hierarchical-models/

In [2]:
df = pd.read_csv("mlr_data.csv")
df[df["Team"] == "arsenal fc"][0:5]


,Team,Squad_Size,Average_Age,Number_of_Foreigners,Average_Market_Value,Total_Market_Value,Position,Goal_Difference,Points,Year,Squad_Size_Normalized_by_Avg,Squad_Size_Normalized_by_Max,Average_Age_Normalized_by_Avg,Average_Age_Normalized_by_Max,Number_of_Foreigners_Normalized_by_Avg,Number_of_Foreigners_Normalized_by_Max,Average_Market_Value_Normalized_by_Avg,Average_Market_Value_Normalized_by_Max,Total_Market_Value_Normalized_by_Avg,Total_Market_Value_Normalized_by_Max
2,arsenal fc,37,23.9,29,6.68,247.00,2,51,83,2004,1.112782,0.902439,0.909264,0.838596,1.428571,1.000000,1.928685,0.624883,2.116420,0.745143
22,arsenal fc,37,24.1,30,6.28,232.33,4,37,67,2005,1.122914,0.948718,0.914091,0.851590,1.488834,1.000000,1.726698,0.518580,1.934761,0.639270
41,arsenal fc,38,23.9,32,7.32,278.18,4,28,68,2006,1.051176,0.883721,0.921179,0.856631,1.434978,1.000000,2.026522,0.650089,2.135936,0.667482
63,arsenal fc,35,23.3,30,7.50,262.40,3,43,83,2007,0.944669,0.760870,0.903801,0.826241,1.260504,0.909091,1.734906,0.673250,1.657282,0.604121
83,arsenal fc,41,22.6,31,7.60,311.45,4,31,72,2008,1.109608,0.931818,0.875290,0.792982,1.371681,1.000000,1.565719,0.559235,1.737454,0.694348


In the Linear Regression Model, all of these observations are assumed to be independent even though there may be a clear team effect present. Mixed Effects Models can help us model this.

##### Standard Multiple Linear Regression
- Assumption: All observations are independent

- Each data point provides unique, unrelated information
- Knowing one observation tells you nothing about another
Formula: All errors are independent: Cov(εᵢ, εⱼ) = 0 for all i ≠ j
Problem with your data: Arsenal 2004 and Arsenal 2005 are NOT independent! They share the same management, stadium, fan base, club culture, etc.

##### Mixed Effects Model
- Assumption: Observations within groups are correlated, but groups are independent

- Data points from the same team are related (correlated errors)
- Data points from different teams are independent
- Formula: Observations share a common team effect: yᵢⱼ = (β₀ + uⱼ) + β₁X₁ᵢⱼ + ... + εᵢⱼ

- uⱼ = team-specific random effect (shared by all observations from team j)
- εᵢⱼ = observation-specific error

What this means: Arsenal's 2004-2008 points are correlated because they share Arsenal's "team effect," but Arsenal and Chelsea are independent.

### Mixed Effects Model with Team

In [3]:
# STEP 1: FIT MODEL ON ALL AVAILABLE DATA

# Filter teams with at least 3 observations for stable estimates
MIN_OBS = 0
team_counts = df.groupby('Team').size()
teams_sufficient = team_counts[team_counts >= MIN_OBS].index
df_train = df[df['Team'].isin(teams_sufficient)].copy()

print(f"Teams used for training: {df_train['Team'].nunique()}")
print(f"Observations used: {len(df_train)}")

# Fit the model
model = smf.mixedlm(
    "Points ~ Squad_Size_Normalized_by_Avg + Average_Market_Value_Normalized_by_Avg",
    data=df_train,
    groups=df_train["Team"]
)

result = model.fit(reml=True, method=['lbfgs', 'powell'])
print(result.summary())


Teams used for training: 44
Observations used: 420
                      Mixed Linear Model Regression Results
Model:                     MixedLM         Dependent Variable:         Points    
No. Observations:          420             Method:                     REML      
No. Groups:                44              Scale:                      85.4472   
Min. group size:           1               Log-Likelihood:             -1534.5406
Max. group size:           21              Converged:                  Yes       
Mean group size:           9.5                                                   
---------------------------------------------------------------------------------
                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------
Intercept                              22.206    4.113  5.399 0.000 14.145 30.266
Squad_Size_Normalized_by_Avg            6.763    3.907  1.731 0.083 -

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


In [4]:
# STEP 2: EXTRACT FIXED EFFECTS (for new teams)

fixed_effects = result.fe_params
print("FIXED EFFECTS (Population Average)")
for param, value in fixed_effects.items():
    print(f"{param:45s}: {value:.4f}")

FIXED EFFECTS (Population Average)
Intercept                                    : 22.2056
Squad_Size_Normalized_by_Avg                 : 6.7630
Average_Market_Value_Normalized_by_Avg       : 23.1198


In [5]:
def predict_points(team_name, squad_size_norm, market_value_norm, model_result):
    """
    Predict points for a team.
    
    Parameters:
    -----------
    team_name : str
        Name of the team (e.g., 'sunderland afc')
    squad_size_norm : float
        Normalized squad size
    market_value_norm : float
        Normalized average market value
    model_result : statsmodels result object
        Fitted mixed effects model
    
    Returns:
    --------
    dict with prediction and method used
    """
    
    # Get fixed effects
    intercept = model_result.fe_params['Intercept']
    beta_squad = model_result.fe_params['Squad_Size_Normalized_by_Avg']
    beta_market = model_result.fe_params['Average_Market_Value_Normalized_by_Avg']
    
    # Fixed effects prediction (baseline)
    pred_fixed = intercept + beta_squad * squad_size_norm + beta_market * market_value_norm
    
    # Check if team has random effect (was in training data)
    random_effects = model_result.random_effects
    
    if team_name.lower() in random_effects:
        # Team was in training data - use random effect
        random_intercept = random_effects[team_name.lower()]['Group']
        pred_total = pred_fixed + random_intercept
        method = "With team effect (in training data)"
        uncertainty = "Lower"
    else:
        # New team - use only fixed effects (population average)
        pred_total = pred_fixed
        random_intercept = 0.0
        method = "Population average (new team)"
        uncertainty = "Higher"
    
    return {
        'team': team_name,
        'predicted_points': pred_total,
        'fixed_effects_only': pred_fixed,
        'team_effect': random_intercept,
        'method': method,
        'uncertainty': uncertainty,
        'squad_size': squad_size_norm,
        'market_value': market_value_norm
    }


In [6]:
# ============================================================
# STEP 4: EXAMPLE PREDICTIONS FOR 2025-26 SEASON
# ============================================================

raw_data = pd.read_csv("transfermarkt_data_raw.csv")

# 1. Get raw data only for the 2025-26 season
df_2025 = raw_data[raw_data["Year"] == 2025].copy()

# 2. Apply normalizations since our first models take normalized features. 
avg_per_year = df_2025.groupby('Year')['Squad_Size'].transform('mean')
df_2025['Squad_Size_Normalized_by_Avg'] = df_2025['Squad_Size'] / avg_per_year

avg_per_year = df_2025.groupby('Year')['Average_Market_Value'].transform('mean')
df_2025['Average_Market_Value_Normalized_by_Avg'] = df_2025['Average_Market_Value'] / avg_per_year

# 3. Keep only the needed columns
df_2025 = df_2025[['Team', 'Squad_Size_Normalized_by_Avg', 'Average_Market_Value_Normalized_by_Avg']]


print("\n" + "="*60)
print("PREDICTIONS FOR 2025-26 PREMIER LEAGUE SEASON")
print("="*60)

predictions = []

for idx, row in df_2025.iterrows():
    team_name = row['Team']
    squad_norm = row['Squad_Size_Normalized_by_Avg']
    market_norm = row['Average_Market_Value_Normalized_by_Avg']
    
    pred = predict_points(team_name, squad_norm, market_norm, result)
    predictions.append(pred)

# Create predictions dataframe
pred_df = pd.DataFrame(predictions)

# Sort by predicted points (highest to lowest)
pred_df = pred_df.sort_values('predicted_points', ascending=False).reset_index(drop=True)

# Add predicted position
pred_df['predicted_position'] = range(1, len(pred_df) + 1)

# Display results
print("\nPredicted 2025-26 Premier League Table:")
print("="*60)
print(pred_df[['predicted_position', 'team', 'predicted_points', 'team_effect', 'method']].to_string(index=False))


PREDICTIONS FOR 2025-26 PREMIER LEAGUE SEASON

Predicted 2025-26 Premier League Table:
 predicted_position                    team  predicted_points  team_effect                              method
                  1              arsenal fc         86.051167     2.724369 With team effect (in training data)
                  2         manchester city         76.701100    -0.907569 With team effect (in training data)
                  3            liverpool fc         75.818773     2.587203 With team effect (in training data)
                  4              chelsea fc         61.821514    -4.030705 With team effect (in training data)
                  5       tottenham hotspur         61.049939    -0.161517 With team effect (in training data)
                  6       manchester united         58.898024     1.442681 With team effect (in training data)
                  7        newcastle united         53.389676    -0.118189 With team effect (in training data)
                  8     

In [7]:
# ============================================================
# ADD CONFIDENCE INTERVALS
# ============================================================

print("\n" + "="*60)
print("PREDICTIONS WITH 95% CONFIDENCE INTERVALS")
print("="*60)

# Calculate standard errors
between_team_var = result.cov_re.iloc[0, 0]  # τ²
within_team_var = result.scale               # σ²

se_new_team = np.sqrt(within_team_var + between_team_var)
se_known_team = np.sqrt(within_team_var)

# Add confidence intervals
for pred in predictions:
    if pred['uncertainty'] == 'Higher':
        se = se_new_team
    else:
        se = se_known_team
    
    pred['ci_lower'] = pred['predicted_points'] - 1.96 * se
    pred['ci_upper'] = pred['predicted_points'] + 1.96 * se
    pred['se'] = se

# Update dataframe with CIs
pred_df_full = pd.DataFrame(predictions)
pred_df_full = pred_df_full.sort_values('predicted_points', ascending=False).reset_index(drop=True)
pred_df_full['predicted_position'] = range(1, len(pred_df_full) + 1)

print(pred_df_full[['predicted_position', 'team', 'predicted_points', 'ci_lower', 'ci_upper', 'uncertainty']].to_string(index=False))



PREDICTIONS WITH 95% CONFIDENCE INTERVALS
 predicted_position                    team  predicted_points  ci_lower   ci_upper uncertainty
                  1              arsenal fc         86.051167 67.933390 104.168943       Lower
                  2         manchester city         76.701100 58.583323  94.818876       Lower
                  3            liverpool fc         75.818773 57.700996  93.936549       Lower
                  4              chelsea fc         61.821514 43.703738  79.939290       Lower
                  5       tottenham hotspur         61.049939 42.932162  79.167715       Lower
                  6       manchester united         58.898024 40.780248  77.015800       Lower
                  7        newcastle united         53.389676 35.271900  71.507452       Lower
                  8             aston villa         52.832279 34.714503  70.950056       Lower
                  9       nottingham forest         49.835283 31.717507  67.953060       Lower
       

In [8]:
# ============================================================
# SUMMARY STATISTICS
# ============================================================

print("\n" + "="*60)
print("PREDICTION SUMMARY")
print("="*60)
print(f"Predicted Champion: {pred_df_full.iloc[0]['team']} ({pred_df_full.iloc[0]['predicted_points']:.1f} points)")
print(f"Predicted Bottom Team: {pred_df_full.iloc[-1]['team']} ({pred_df_full.iloc[-1]['predicted_points']:.1f} points)")
print(f"\nAverage predicted points: {pred_df_full['predicted_points'].mean():.1f}")
print(f"Std dev of predictions: {pred_df_full['predicted_points'].std():.1f}")

# Teams with team effects vs population average
n_with_effect = (pred_df_full['uncertainty'] == 'Lower').sum()
n_population = (pred_df_full['uncertainty'] == 'Higher').sum()
print(f"\nTeams with historical effect: {n_with_effect}")
print(f"Teams using population average: {n_population}")


PREDICTION SUMMARY
Predicted Champion: arsenal fc (86.1 points)
Predicted Bottom Team: sunderland afc (37.2 points)

Average predicted points: 52.4
Std dev of predictions: 13.7

Teams with historical effect: 20
Teams using population average: 0


In [11]:
df

,Team,Squad_Size,Average_Age,Number_of_Foreigners,Average_Market_Value,Total_Market_Value,Position,Goal_Difference,Points,Year,Squad_Size_Normalized_by_Avg,Squad_Size_Normalized_by_Max,Average_Age_Normalized_by_Avg,Average_Age_Normalized_by_Max,Number_of_Foreigners_Normalized_by_Avg,Number_of_Foreigners_Normalized_by_Max,Average_Market_Value_Normalized_by_Avg,Average_Market_Value_Normalized_by_Max,Total_Market_Value_Normalized_by_Avg,Total_Market_Value_Normalized_by_Max
0,chelsea fc,31,24.9,24,10.69,331.48,1,57,95,2004,0.932331,0.756098,0.947308,0.873684,1.182266,0.827586,3.086473,1.000000,2.840287,1.000000
1,manchester united,37,24.7,25,7.93,293.23,3,32,77,2004,1.112782,0.902439,0.939699,0.866667,1.231527,0.862069,2.289591,0.741815,2.512542,0.884608
2,arsenal fc,37,23.9,29,6.68,247.00,2,51,83,2004,1.112782,0.902439,0.909264,0.838596,1.428571,1.000000,1.928685,0.624883,2.116420,0.745143
3,liverpool fc,38,25.3,26,5.85,222.13,5,11,58,2004,1.142857,0.926829,0.962526,0.887719,1.280788,0.896552,1.689043,0.547240,1.903322,0.670116
4,tottenham hotspur,38,25.3,22,3.41,129.45,9,6,52,2004,1.142857,0.926829,0.962526,0.887719,1.083744,0.758621,0.984553,0.318990,1.109193,0.390521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,everton fc,39,25.8,21,9.31,363.23,13,-2,48,2024,0.972569,0.672414,1.015348,0.921429,0.876827,0.636364,0.599311,0.301685,0.574897,0.267081
416,fulham fc,28,27.7,21,12.94,362.35,11,0,54,2024,0.698254,0.482759,1.090122,0.989286,0.876827,0.636364,0.832985,0.419313,0.573505,0.266434
417,southampton fc,42,25.3,23,7.38,310.05,20,-60,12,2024,1.047382,0.724138,0.995671,0.903571,0.960334,0.696970,0.475072,0.239145,0.490728,0.227978
418,ipswich town,35,26.6,17,7.72,270.10,19,-46,22,2024,0.872818,0.603448,1.046832,0.950000,0.709812,0.515152,0.496958,0.250162,0.427497,0.198603
